In [1]:
from tika import parser 
import re

raw = parser.from_file('paper4.pdf')
text = raw['content']

section_heading = '3. Results and discussion'

# Define the regular expression pattern to find the section heading
pattern = re.compile(section_heading, re.IGNORECASE)

match = pattern.search(text)
print(match)
if match:
    # Find the end of the section
    end_match = re.search(r'(Introduction|Background|Methods|Results|Discussion|Conclusion|Summary)', text[match.end():], re.MULTILINE)
    if end_match:
        section_text = text[match.end():end_match.start() + match.end()]
    else:
        section_text = text[match.end():]
else:
    section_text = ''

print(section_text)

2023-03-31 10:02:40,748 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-03-31 10:02:45,764 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


<re.Match object; span=(34459, 34484), match='3. Results and discussion'>
 

3.1. PM2.5 mass concentration 

The PM2.5(eq) concentration is the sum of NR-PM2.5 (Org, SO4
2− , NO3

− , 
NH4

+ and Cl− ), elements and BC and their hourly variation is shown in 
Fig. 1 (a). For the common period, the hourly average PM2.5(eq), NR- 
PM2.5, PMEl and BC are 28.7 ± 13.2 (8–94) μg.m− 3, 14.1 ± 8.1 (3–49) μg. 
m− 3, 9.7 ± 9.8 (1–87) μg.m− 3 and 5 ± 3.9 (0.7–29) μg.m− 3, respectively. 
The PM2.5(eq) mass includes elements, organics, inorganics, and BC 
representing 34%, 28%, 21% and 17%, respectively. The hourly PM2.5 

(eq) concentration correlates strongly with the EBAM PM2.5 (slope = 0.51 
and R = 0.8) (Fig. S1). The time series is dominated by elements as 
several dust storms characterized by high Si, Al, and Fe concentrations 
occurred during the campaign (June 11th, 16th, 17th, and 21st, 2020). 
Organics dominate the NR-PM2.5 concentration, followed by sulfate with 
a combined contribution o

In [2]:
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Mihir
[nltk_data]     Nandawat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
section_text = section_text.replace("\n","")
sentences = tokenize.sent_tokenize(section_text)

f = open("sentences/sentences_paper4.txt", "w", encoding="utf-8")
mylist = []
for s in sentences:
        mylist.append(s.strip())
for i in mylist:
    f.write(i)
    f.write("\n")
f.close()

In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import re
from chemdataextractor import Document
from IPython.display import clear_output

In [5]:
f = open("sentences/sentences_paper4.txt", encoding="utf8")
sentences = []

for i in f:
    sentences.append(i.strip())

f.close()

In [6]:
def valid_xml_char_ordinal(c):
    codepoint = ord(c)
    # conditions ordered by presumed frequency
    return (
        0x20 <= codepoint <= 0xD7FF or
        codepoint in (0x9, 0xA, 0xD) or
        0xE000 <= codepoint <= 0xFFFD or
        0x10000 <= codepoint <= 0x10FFFF
        )

In [7]:
short = []
for i in range(len(sentences)):
    if(len(sentences[i]) < 10):
        short.append(sentences[i])

for i in short:
    sentences.remove(i)

In [8]:
filter_sentences = []
filter_index = []
for i in range(len(sentences)):
    cleaned_string = ''.join(c for c in sentences[i] if valid_xml_char_ordinal(c))
    
    if_num = any(chr.isdigit() for chr in cleaned_string)
    if_emp = False
    if_long = True
    doc = Document(cleaned_string)
    
    if(len(doc.cems) != 0):
        if_emp = True
        
    if(len(cleaned_string) < 10):
        if_long = False
        
    if((if_num or if_emp) and if_long):
        filter_sentences.append(sentences[i])
        filter_index.append(i)

In [9]:
filter_sentences

['PM2.5 mass concentration The PM2.5(eq) concentration is the sum of NR-PM2.5 (Org, SO42− , NO3− , NH4+ and Cl− ), elements and BC and their hourly variation is shown in Fig.',
 'For the common period, the hourly average PM2.5(eq), NR- PM2.5, PMEl and BC are 28.7 ± 13.2 (8–94) μg.m− 3, 14.1 ± 8.1 (3–49) μg.',
 'm− 3, 9.7 ± 9.8 (1–87) μg.m− 3 and 5 ± 3.9 (0.7–29) μg.m− 3, respectively.',
 'The PM2.5(eq) mass includes elements, organics, inorganics, and BC representing 34%, 28%, 21% and 17%, respectively.',
 'The hourly PM2.5 (eq) concentration correlates strongly with the EBAM PM2.5 (slope = 0.51 and R = 0.8) (Fig.',
 'The time series is dominated by elements as several dust storms characterized by high Si, Al, and Fe concentrations occurred during the campaign (June 11th, 16th, 17th, and 21st, 2020).',
 'Organics dominate the NR-PM2.5 concentration, followed by sulfate with a combined contribution of 83% to the total NR-PM2.5.',
 'Sulfate has a less diurnal variation (Fig.',
 '1 (b)) a

In [10]:
tag = [0]*(len(sentences))
for i in filter_index:
    tag[i] = 1

In [11]:
df = pd.DataFrame(list(zip(sentences,tag)), columns = ['sentence','tag'])

In [13]:
df.tail()

,sentence,tag
198,The difference between the percentage composit...,1
199,The source region predicted by the CWT model f...,1
200,The Dust and S-rich factors in percentage sour...,1
201,The contribution of the Cl-rich factor has dec...,1
202,"The possible source region of Cu-rich, SFC-I a...",1


In [14]:
for i in range(len(df)):
    if(df.iloc[[i]]['tag'][i] == 1):
        print(df.iloc[[i]]['sentence'][i])
        print('\n')
        print('Retag?')
        inp = str(input())
        
        if(inp == 'y'):
            df.iat[i,1] = 0
            clear_output()
        else:
            clear_output()
            #continue

In [15]:
df.to_csv('data/paper4.csv')

In [19]:
df['tag'].value_counts()

0    130
1     73
Name: tag, dtype: int64